In [67]:
import pandas as pd
import json

In [68]:
# load table & mobie dataset
table = pd.read_csv('categories.csv', sep='\t', dtype=str)  # type: ignore
with open('/home/hellgoth/Documents/work/projects/culture-collections_project/culture-collections/data/volumes/dataset.json', 'r') as f:
    dataset = json.load(f)

In [69]:
table

,Clade according figure 1,new_filename_iteration5_afterEverythingElse_justDeconvAlign,Cell cycle stage,Spools,Cortical centrin,Apical centrin ring,Localisation
0,Glaucophyta,rcc6731_pfa_dna-tub-nhs_20230616_od_s00,mitotic onset,No,-,-,Central
1,Glaucophyta,rcc6731_pfa_dna-tub-nhs_20230616_od_s01,Mitosis,No,-,-,Central
2,Glaucophyta,rcc6731_pfa_dna-tub-nhs_20230616_od_s02,Interphase,No,-,-,Central
3,Glaucophyta,rcc6731_pfa_dna-tub-nhs_20230616_od_s03,Interphase,No,-,-,Central
4,Glaucophyta,rcc6731_pfa_dna-tub-nhs_20230616_od_s04,Interphase,No,-,-,Central
...,...,...,...,...,...,...,...
1762,NaN,rcc696rcc10662_pfa_dna-tub-nhs-na_20230720_csd_01,Interphase,No,-,-,Central
1763,NaN,rcc696rcc10662_pfa_dna-tub-nhs_20230728_csd_01,Interphase,No,-,-,Central
1764,NaN,rccp2-b1_pfa_dna-cetn-nhs-tub_20240209_od_s00,Interphase,No,No,No,Central
1765,NaN,rccp2-b1_pfa_dna-cetn-nhs-tub_20240209_od_s01,Interphase,No,No,No,Central


In [70]:
table.describe()

,Clade according figure 1,new_filename_iteration5_afterEverythingElse_justDeconvAlign,Cell cycle stage,Spools,Cortical centrin,Apical centrin ring,Localisation
count,1733,1757,1767,1579,1711,1712,832
unique,10,1757,7,3,3,4,5
top,Alveolata,rcc6731_pfa_dna-tub-nhs_20230616_od_s00,Interphase,No,-,-,Central
freq,674,1,1390,1524,927,936,437


In [71]:
table.loc[table["new_filename_iteration5_afterEverythingElse_justDeconvAlign"] == "vf21108_pfa_tub-cetn_20251803_cl_03"]

,Clade according figure 1,new_filename_iteration5_afterEverythingElse_justDeconvAlign,Cell cycle stage,Spools,Cortical centrin,Apical centrin ring,Localisation
1157,Alveolata,vf21108_pfa_tub-cetn_20251803_cl_03,unknown,No,Yes,-,NaN


In [72]:
table["Clade according figure 1"].value_counts()

Clade according figure 1
Alveolata        674
Viridiplantae    328
Haptophyta       175
Stramenopiles    172
Cryptista        141
Environmental    117
Discoba           94
Rhodophyta        25
Glaucophyta        5
Rhizaria           2
Name: count, dtype: int64

In [73]:
table["Spools"].value_counts()

Spools
No       1524
Yes        54
Maybe       1
Name: count, dtype: int64

In [74]:
table["Cortical centrin"].value_counts()

Cortical centrin
-      927
No     644
Yes    140
Name: count, dtype: int64

In [75]:
table["Apical centrin ring"].value_counts()

Apical centrin ring
-      936
No     700
Yes     75
.        1
Name: count, dtype: int64

In [76]:
table["Cell cycle stage"].value_counts()

Cell cycle stage
Interphase             1390
unknown                 269
Mitosis                  79
mitotic onset             9
Interphase, Mitosis       9
Meiosis?                  7
Cyst                      4
Name: count, dtype: int64

In [77]:
table["Localisation"].value_counts()

Localisation
Central                    437
NE                         194
Distributed                 98
No or insufficient data     81
Variable                    22
Name: count, dtype: int64

In [78]:
new_dataset = dataset.copy()
stages = {}
for view_name, view in new_dataset["views"].items():
    row = table.loc[table["new_filename_iteration5_afterEverythingElse_justDeconvAlign"] == view_name]
    if not row.empty:
        stage = row["Cell cycle stage"].values[0]
        if stage not in stages:
            stages[stage] = 0
        stages[stage] += 1
stages

{'Interphase': 1357,
 'unknown': 170,
 'Mitosis': 76,
 'mitotic onset': 8,
 'Interphase, Mitosis': 8,
 'Meiosis?': 7,
 'Cyst': 4}

In [83]:
# iterate over all views and change to selection groups, add categories
new_dataset = dataset.copy()
for view_name, view in new_dataset["views"].items():
    view["uiSelectionGroups"] = ["all_volumes"]
    if "uiSelectionGroup" in view:
        del view["uiSelectionGroup"]
    if view_name == "default":
        continue

    row = table.loc[table["new_filename_iteration5_afterEverythingElse_justDeconvAlign"] == view_name]
    if not row.empty:
        clade = row["Clade according figure 1"].values[0]
        if clade != "Glaucophyta" and clade != "Rhizaria" and not pd.isna(clade):
            view["uiSelectionGroups"].append(clade.lower())
        if row["Spools"].values[0] == "Yes":
            view["uiSelectionGroups"].append("spools")
        if row["Cortical centrin"].values[0] == "Yes":
            view["uiSelectionGroups"].append("cortical_centrin")
        if row["Apical centrin ring"].values[0] == "Yes":
            view["uiSelectionGroups"].append("apical_centrin_ring")
        stage = row["Cell cycle stage"].values[0]
        if stage == "Mitosis" or stage == "mitotic onset" or stage == "Interphase, Mitosis":
            view["uiSelectionGroups"].append("mitosis")
        loc = row["Localisation"].values[0]
        if loc == "Central":
            view["uiSelectionGroups"].append("nucleolus_central")
        if loc == "NE":
            view["uiSelectionGroups"].append("nucleolus_at_nuclear_envelope")
        

In [84]:
"default" in new_dataset["views"]

True

In [85]:
# save to file and test
with open('/home/hellgoth/Documents/work/projects/culture-collections_project/culture-collections/data/volumes_with_categories/dataset.json', 'w') as f:
    json.dump(new_dataset, f, indent=2)

In [86]:
# save to file and test
with open('/home/hellgoth/Documents/work/projects/culture-collections_project/culture-collections/data/volumes/dataset.json', 'w') as f:
    json.dump(new_dataset, f, indent=2)